# Installing COLMAP on Google Colab

COLMAP is a popular 3D reconstruction software. It takes multiple images of a scene and reconstructs the 3D structure of that scene along with the camera positions. It doesn Feature Detection & Matching, Structure-from-Motion (SfM), and Multi-View Stereo (MVS).

## Install Dependencies

In [ ]:
!sudo apt update
!sudo apt install -y \
    build-essential cmake git \
    libboost-all-dev libeigen3-dev libsuitesparse-dev \
    qtbase5-dev libglew-dev libglfw3-dev \
    libx11-dev libopencv-dev libgoogle-glog-dev \
    libgflags-dev libatlas-base-dev libopencv-core-dev \
    libopenimageio-dev openimageio-tools libopenexr-dev \
    libcgal-dev libcgal-qt5-dev libmetis-dev

### Install Abseil
Abseil is a collection of open-source C++ libraries from Google. Ceres depends on Abseil for some of its internal utilities like containers, flags, or threading helpers.

In [ ]:
%cd /content
!git clone https://github.com/abseil/abseil-cpp.git
%cd /content/abseil-cpp
!git checkout 20230802.0

!rm -rf build
!mkdir build
%cd build
!cmake .. \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=/usr/local \
  -DCMAKE_POSITION_INDEPENDENT_CODE=ON
!make -j$(nproc)
!sudo make install

### Install Ceres Solver using the installed Abseil
Ceres is an open-source C++ library for non-linear least squares optimization. COLMAP uses Ceres to optimize camera poses, 3D points, and bundle adjustment.

In [ ]:
%cd /content
!git clone https://github.com/ceres-solver/ceres-solver.git
%cd /content/ceres-solver
!git checkout 2.1.0

!rm -rf build
!mkdir build
%cd build
!cmake .. \
  -DBUILD_TESTING=OFF \
  -DBUILD_EXAMPLES=OFF \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=/usr/local
!make -j$(nproc)
!sudo make install

## Install COLMAP (CPU-only)

In [ ]:
%cd /content
!git clone https://github.com/colmap/colmap.git
%cd colmap

!rm -rf build
!mkdir build
%cd build
!cmake .. \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_INSTALL_PREFIX=/usr/local \
    -DCUDA_ENABLED=OFF \
    -DCeres_DIR=/usr/local/lib/cmake/Ceres \
    -DAbsl_DIR=/usr/local/lib/cmake/absl

!make -j$(nproc)
!sudo make install

### Test COLMAP installation

In [ ]:
!colmap --help

## Run COLMAP (CPU) on your images
Folder structure for your images have to be like this:

```
/content/project/
├── images/        # your input images
├── database.db
└── sparse/

```

You can put images in your Google Drive, ```image_source``` folder, and change the path below. This command will copy images from the ```image_source``` folder on your Google Drive into your Colab environment.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content
!rm -rf project
!mkdir -p project/images
!cp -r "/content/drive/MyDrive/image_source/." "/content/project/images"

### Start Feature Extraction

If your images are from different cameras, set ```ImageReader.single_camera``` to ```0```, else set to ```1```.

In [ ]:
!export QT_QPA_PLATFORM=offscreen
!colmap feature_extractor \
  --database_path /content/project/database.db \
  --image_path /content/project/images \
  --ImageReader.single_camera 0 \
  --FeatureExtraction.use_gpu 0

### Feature matching (exhaustive)

In [ ]:
!colmap exhaustive_matcher \
  --database_path /content/project/database.db \
  --FeatureMatching.use_gpu 0

### Sparse reconstruction

In [ ]:
!mkdir -p /content/project/sparse
!colmap mapper \
  --database_path /content/project/database.db \
  --image_path /content/project/images \
  --output_path /content/project/sparse

### Export undistorted COLMAP reconstruction

In [ ]:
!colmap image_undistorter \
    --image_path /content/project/images \
    --input_path /content/project/sparse/0 \
    --output_path /content/project/undistorted \
    --output_type COLMAP \
    --max_image_size 1600

### Copy COLMAP results to Google Drive

In [ ]:
!mkdir "/content/drive/MyDrive/COLMAP Results"
!cp -r "/content/project" "/content/drive/MyDrive/COLMAP Results"